In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import os
  CWD = '/content/drive/MyDrive/DataSources/SkinCare'
  # G_path = '/content/drive/MyDrive/DataSources/SkinCare_Project/SkinCare_Project/Project_Data'
  os.chdir(CWD)
except:None
G_path = './Project_Data'

In [2]:
import numpy as np
import os
import torch
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from features.usefull_functions import *
from features.NETs import *
from features.Model_Training import *
from statistics import mean
import sklearn as sk
import warnings
from sklearn.utils import resample
import pickle as pickle
import PIL as pl
import pandas as pd
from PIL.Image import Transpose
from PIL import Image
warnings.filterwarnings('ignore')

# **Import Augmented Images and rescaling to 280x210**

In [3]:
Augment_Path = Augment_Path = G_path + '/' + '01_Augmented_Imaged'
img_list = []
for path, dirs, files in  os.walk(Augment_Path):
    for f in files:
        img_list.append( path +'/' + f)
    for d in dirs:
        img_list.append( path + d)
img_list = list(set([x for x in img_list if ".jpg" in x] ))

Images_280x210 = G_path + '/04_Rescaled_Images'

try:
    os.makedirs(Images_280x210)
except:
    None

for img in tqdm(img_list):
    file_out = Images_280x210 +'/'+ re.findall('\w+',img)[-2:-1][0] +'.jpg'
    os.system('convert %s %s %s' % ('-resize 46.66%',img,file_out))

100%|██████████| 4647/4647 [01:36<00:00, 47.99it/s]


# **Import Rescaled Images and convert to arrays**

In [4]:
img_list = os.listdir(Images_280x210)
Augmented_280x210_dataset = pd.DataFrame(columns=['image_id','img_array'])
for img in tqdm(img_list):
    if '.jpg' in img:
        img_name = img.replace('.jpg','')
        image = pl.Image.open( Images_280x210 +'/'+ img)
        img_array = np.asarray(image)
        new_row = pd.Series({'image_id':img_name,'img_array':img_array}, name='')
        Augmented_280x210_dataset = Augmented_280x210_dataset.append(new_row)

100%|██████████| 4648/4648 [00:08<00:00, 573.37it/s]


In [5]:
Meta_Data_Augmented = pickle.load(open(G_path + '/02_Augmented_MetaData/'+ '/Meta_Data_Augmented','rb'))
blc_test_meta = pickle.load(open(G_path + '/02_Augmented_MetaData/'+ '/balanced_test_set','rb'))

CNN_280x210_dataset = Augmented_280x210_dataset.merge(Meta_Data_Augmented, how="left",on='image_id')
blc_test_dataset_280x210 = Augmented_280x210_dataset[['image_id','img_array']].merge(blc_test_meta, how="right",on='image_id')

# **CNN_280x210: Standardization**

In [6]:
df_ = CNN_280x210_dataset
train_data = df_[df_.type=='train']
valid_data = df_[df_.type=='valid']
test_data = df_[df_.type=='test']
print('Train size: ',train_data.shape[0] ,' Valid Size: ',valid_data.shape[0], ' Test size: ', test_data.shape[0])

Train size:  3833  Valid Size:  407  Test size:  407


In [7]:
training_data = pd.concat([train_data,valid_data], axis=0)
training_arrays_scaled = np.stack(training_data['img_array'].values)  / 255
Mean = training_arrays_scaled.mean(axis = (0,1,2)) 
STD = training_arrays_scaled.std(axis = (0,1,2))
print(f"Mean : {Mean}   STD: {STD}") 

Mean : [0.76570269 0.55224556 0.57657015]   STD: [0.13616887 0.1481389  0.16343116]


In [8]:
#Train data standardization
train_arrays_std = np.stack(train_data['img_array'].values) / 255
for i in range(0,train_arrays_std.shape[0]):
    train_arrays_std[i] = (train_arrays_std[i] - Mean) / STD
#Validation data standardization
valid_arrays_std = np.stack(valid_data['img_array'].values) / 255
for i in range(0,valid_arrays_std.shape[0]):
    valid_arrays_std[i] = (valid_arrays_std[i] - Mean) / STD
#Test data standardization
test_arrays_std = np.stack(test_data['img_array'].values) / 255
for i in range(0,test_arrays_std.shape[0]):
    test_arrays_std[i] = (test_arrays_std[i] - Mean) / STD

#Test-Valid data standardization
test_valid_arrays_std = np.stack(blc_test_dataset_280x210['img_array'].values) / 255
for i in range(0,test_valid_arrays_std.shape[0]):
    test_valid_arrays_std[i] = (test_valid_arrays_std[i] - Mean) / STD

In [9]:
Mean_t = test_arrays_std.mean(axis = (0,1,2)) 
STD_t = test_arrays_std.std(axis = (0,1,2))
print(f"Test Set Mean : {Mean_t}   Test Set STD: {STD_t}") 

Test Set Mean : [-0.01839491 -0.04231122 -0.04687724]   Test Set STD: [0.96394774 0.99849143 1.00558878]


In [10]:
metda_keys = train_data[['type','image_id']].reset_index(drop=True), valid_data[['type','image_id']].reset_index(drop=True), test_data[['type','image_id']].reset_index(drop=True)

train_arrays_std_T = torch.tensor(train_arrays_std, dtype= torch.float32).transpose(3,1)
train_labels = torch.tensor(train_data['image_label'].values.astype(np.long))

valid_arrays_std_T = torch.tensor(valid_arrays_std, dtype= torch.float32).transpose(3,1)
valid_labels = torch.tensor(valid_data['image_label'].values.astype(np.long))

test_arrays_std_T = torch.tensor(test_arrays_std, dtype= torch.float32).transpose(3,1)
test_labels = torch.tensor(test_data['image_label'].values.astype(np.long))

test_valid_arrays_std_T = torch.tensor(test_valid_arrays_std, dtype= torch.float32).transpose(3,1)
test_valid_labels = torch.tensor(blc_test_dataset_280x210['image_label'].values.astype(np.long))

train_labels_binary =  torch.tensor(label_to_binary(train_labels))
valid_labels_binary =  torch.tensor(label_to_binary(valid_labels))
test_labels_binary =  torch.tensor(label_to_binary(test_labels))
test_valid_labels_binary =  torch.tensor(label_to_binary(test_valid_labels))


input_data = (train_arrays_std_T , valid_arrays_std_T, test_arrays_std_T ,metda_keys)
labels = ((train_labels,valid_labels,test_labels), (train_labels_binary , valid_labels_binary ,test_labels_binary))

balc_evaluation_set = (test_valid_arrays_std_T,test_valid_labels,test_valid_labels_binary)

In [11]:
dataset_training = G_path + '/06_Rescaled_DataSet'

try:
    os.makedirs(dataset_training)
except:
    None

pickle.dump((input_data,labels), open(dataset_training + '/Input_DataSet_280x210', 'wb'))
pickle.dump(balc_evaluation_set, open(dataset_training + '/Balanced_Test_Set_280x210', 'wb'))

# **Import AutoEncoders Images**

In [12]:
def import_imges(img_path):
    img_list = []
    for path, dirs, files in  os.walk(img_path):
        for f in files:
            img_list.append( path +'/' + f)
        for d in dirs:
            img_list.append( path + d)
    img_list = list(set([x for x in img_list if ".jpg" in x] ))

    dataset = pd.DataFrame(columns=['image_id','img_array'])
    for img in tqdm(img_list):
        img_name = re.findall('\w+', img)[-2:-1][0]
        image = pl.Image.open( img)
        img_array = np.asarray(image)
        new_row = pd.Series({'image_id':img_name,'img_array':img_array}, name='')
        dataset = dataset.append(new_row)
    return dataset

train_path = G_path + '/Augmented_AutoEncoded/train'
valid_path = G_path + '/Augmented_AutoEncoded/valid'
test_path = G_path + '/Augmented_AutoEncoded/test'

train_set = import_imges(train_path)
valid_set = import_imges(valid_path)
test_set = import_imges(test_path)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [13]:
dataset_ = pd.concat([train_set,valid_set,test_set],axis=0)
A_E_dataset = dataset_.merge(Meta_Data_Augmented, how='left', on='image_id')
blc_test_dataset_A_E = dataset_[['image_id','img_array']].merge(blc_test_meta, how="right",on='image_id')

In [14]:
df_ = A_E_dataset
train_data = df_[df_.type=='train']
valid_data = df_[df_.type=='valid']
test_data = df_[df_.type=='test']
print('Train size: ',train_data.shape[0] ,' Valid Size: ',valid_data.shape[0], ' Test size: ', test_data.shape[0])

Train size:  0  Valid Size:  0  Test size:  0


In [16]:
training_data = pd.concat([train_data,valid_data], axis=0)
training_arrays_scaled = np.stack(training_data['img_array'].values)  / 255
Mean = training_arrays_scaled.mean(axis = (0,1,2)) 
STD = training_arrays_scaled.std(axis = (0,1,2))
print(f"Mean : {Mean}   STD: {STD}") 

ValueError: need at least one array to stack

In [ ]:
#Train data standardization
train_arrays_std = np.stack(train_data['img_array'].values) / 255
for i in range(0,train_arrays_std.shape[0]):
    train_arrays_std[i] = (train_arrays_std[i] - Mean) / STD
#Validation data standardization
valid_arrays_std = np.stack(valid_data['img_array'].values) / 255
for i in range(0,valid_arrays_std.shape[0]):
    valid_arrays_std[i] = (valid_arrays_std[i] - Mean) / STD
#Test data standardization
test_arrays_std = np.stack(test_data['img_array'].values) / 255
for i in range(0,test_arrays_std.shape[0]):
    test_arrays_std[i] = (test_arrays_std[i] - Mean) / STD

#Test-Valid data standardization
test_valid_arrays_std = np.stack(blc_test_dataset_A_E['img_array'].values) / 255
for i in range(0,test_valid_arrays_std.shape[0]):
    test_valid_arrays_std[i] = (test_valid_arrays_std[i] - Mean) / STD

In [ ]:
Mean_t = test_arrays_std.mean(axis = (0,1,2)) 
STD_t = test_arrays_std.std(axis = (0,1,2))
print(f"Test Set Mean : {Mean_t}   Test Set STD: {STD_t}") 

Test Set Mean : [-0.04389305 -0.05144053  0.03501551]   Test Set STD: [0.93557654 0.98728975 1.00074642]


In [ ]:
metda_keys = train_data[['type','image_id']].reset_index(drop=True), valid_data[['type','image_id']].reset_index(drop=True), test_data[['type','image_id']].reset_index(drop=True)

In [ ]:
train_arrays_std_T = torch.tensor(train_arrays_std, dtype= torch.float32).transpose(3,1)
train_labels = torch.tensor(train_data['image_label'].values.astype(np.long))

valid_arrays_std_T = torch.tensor(valid_arrays_std, dtype= torch.float32).transpose(3,1)
valid_labels = torch.tensor(valid_data['image_label'].values.astype(np.long))

test_arrays_std_T = torch.tensor(test_arrays_std, dtype= torch.float32).transpose(3,1)
test_labels = torch.tensor(test_data['image_label'].values.astype(np.long))

test_valid_arrays_std_T = torch.tensor(test_valid_arrays_std, dtype= torch.float32).transpose(3,1)
test_valid_labels = torch.tensor(blc_test_dataset_A_E['image_label'].values.astype(np.long))

A_E_input_data = (train_arrays_std_T , valid_arrays_std_T, test_arrays_std_T ,metda_keys)

train_labels_binary =  torch.tensor(label_to_binary(train_labels))
valid_labels_binary =  torch.tensor(label_to_binary(valid_labels))
test_labels_binary =  torch.tensor(label_to_binary(test_labels))

A_E_labels = ((train_labels,valid_labels,test_labels), (train_labels_binary , valid_labels_binary ,test_labels_binary))

A_E_balc_test_set = (test_valid_arrays_std_T,test_valid_labels)

In [ ]:
pickle.dump((A_E_input_data,A_E_labels), open(G_path + '/Augmented_AutoEncoded/'+ 'Latent_Input_Data_labels', 'wb'))
pickle.dump(A_E_balc_test_set, open(G_path +  '/Augmented_AutoEncoded/'+  '/Evaluation_set_AutoEncoder', 'wb'))